In [83]:
import pandas as pd
import numpy as np
import os
import requests
import json


In [84]:
def data_import():
    """
    Import der Daten aus allen Dateien, die mit 'waqi-covid-' anfangen.
    Entfernen von Duplikaten und Umbenennung bestimmter Spaltenwerte.
    Zusammenführung der DataFrames.
    """
    data_folder = './data/'
    all_files = [f for f in os.listdir(data_folder) if f.startswith('waqi-covid-') and f.endswith('.csv') or f == 'airquality-covid19-cities.json']
    dataframes = []

    if not all_files:
        print("Keine Dateien gefunden.")
        return None

    for file in all_files:
        file_path = os.path.join(data_folder, file)
        try:
            df = pd.read_csv(file_path, comment='#')

            if "Specie" not in df.columns:
                print(f"Spalte 'Specie' fehlt in {file}")
                continue

            df["Specie"] = df["Specie"].replace("wind gust", "wind-gust")
            df["Specie"] = df["Specie"].replace("wind speed", "wind-speed")
            df = df.drop_duplicates()

            if df.empty:
                print(f"{file} enthält nach Duplikat-Entfernung keine Daten mehr.")
                continue

            dataframes.append(df)
        except Exception as e:
            print(f"Fehler beim Verarbeiten von {file}: {e}")

    if not dataframes:
        print("Keine gültigen Daten vorhanden.")
        return None

    return pd.concat(dataframes, ignore_index=True)

In [85]:
df=data_import()
print(df)

Spalte 'Specie' fehlt in airquality-covid19-cities.json
waqi-covid-2022Q1.csv enthält nach Duplikat-Entfernung keine Daten mehr.
                Date Country    City Specie  count   min    max  median  \
0         2015-01-06      KR  Jeonju     co    124   0.1   12.3     4.5   
1         2015-01-22      KR  Jeonju     co    116   4.5   10.0     6.7   
2         2015-03-30      KR  Jeonju     co    118   1.2   11.2     5.6   
3         2015-05-27      KR  Jeonju     co     93   2.3    5.6     3.4   
4         2015-02-03      KR  Jeonju     co    133   4.5   13.4     7.8   
...              ...     ...     ...    ...    ...   ...    ...     ...   
14251935  2024-08-12      IR  Tehran    so2    154   6.0  101.0    12.0   
14251936  2025-01-13      IR  Tehran    so2    516  10.0   74.0    19.0   
14251937  2024-02-10      IR  Tehran    so2    299   5.0   76.0    34.0   
14251938  2024-05-16      IR  Tehran    so2    559   6.0   92.0    17.0   
14251939  2025-01-04      IR  Tehran    so2   

In [86]:
def add_geodata(df):
    """
    Fügt die Geodaten zu den Städten hinzu
    """
    df = df.copy()

    # JSON-Datei laden
    with open('./data/airquality-covid19-cities.json', 'r', encoding='utf-8') as file:
        geodata = json.load(file)
    
    geodata = geodata["data"] 

    # Erstellen eines DataFrames mit Städten und Geokoordinaten
    df_places = pd.DataFrame([
        {
            "City": entry["Place"]["name"],  # Stadtname
            "Latitude": entry["Place"]["geo"][0],  
            "Longitude": entry["Place"]["geo"][1]
        }
        for entry in geodata if "Place" in entry])  # Nur Einträge mit "Place" verwenden
    

    # Standardisiere den Stadtnamen für eine bessere Übereinstimmung
    df["City"] = df["City"].str.lower().str.strip()
    df_places["City"] = df_places["City"].str.lower().str.strip()

    # Zusammenführen der beiden DataFrames über "City"
    df = df.merge(df_places, on="City", how="left")

    return df

In [87]:
# def data_cleaning(df):
#     """Bereinigung der Daten
#     - nicht benötigte Spalten löschen
#     - Eine Stadt pro Land mit den meisten Messwerten 
#     - und die Liste als csv ins Datenverzeichnis speichern
#     - Zusammenfassung der Daten nach Datum, Land, Stadt und Spezies, so dass nur ein Messwert je Species (Median) pro Tag/ Stadt verbleibt
#     - filtern des df nach den ausgewählten Städten
#     - Spalte Species aufteilen
#     - df als csv speichern im Datenverzeichnis
#     """
#     df = df.copy()

#     df = df.drop(columns=['variance', 'min', 'max'], errors='ignore')

#     # city_counts = df.groupby(["Country", "City"]).size().reset_index(name="count")
#     # cities = city_counts.loc[city_counts.groupby("Country")["count"].idxmax()]
  
#     # output_path = './data/city_per_country.csv'
#     # os.makedirs(os.path.dirname(output_path), exist_ok=True)
#     # cities.to_csv(output_path, index=False)
#     # print(f"✅ Datei wurde gespeichert: {output_path}")
    
#     # cities=cities['City'].tolist()
#     # df = df[df['City'].isin(cities)]

#     df = df.groupby(["Date", "Country", "City", "Specie"], as_index=False).agg({"median": "mean"})  

#     df = df.pivot(index=["Date", "Country", "City"], columns="Specie", values='median').reset_index()

#     output_path = './data/cleaned_data.csv'
#     os.makedirs(os.path.dirname(output_path), exist_ok=True)
#     df.to_csv(output_path, index=False)
#     print(f"✅ Datei wurde gespeichert: {output_path}")

#     return df



In [88]:
def data_cleaning(df):
    """Bereinigung der Daten
    - nicht benötigte Spalten löschen
    - Zusammenfassung der Daten nach Datum, Land, Stadt und Spezies, so dass nur ein Messwert je Species (Median) pro Tag/ Stadt verbleibt
    - Spalte Species aufteilen
    - df als csv speichern im Datenverzeichnis
    """
    df = df.copy()

    df = df.drop(columns=['variance', 'min', 'max'], errors='ignore')

    df = df.groupby(["Date", "Country", "City", "Specie"], as_index=False).agg({"median": "mean"})  

    df = df.pivot(index=["Date", "Country", "City"], columns="Specie", values='median').reset_index()


    output_path = './data/cleaned_data.csv'
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    df.to_csv(output_path, index=False)
    print(f"✅ Datei wurde gespeichert: {output_path}")


    df= add_geodata(df)


    return df



In [89]:
df=data_cleaning(df)
print(df.shape)
df.head()  

✅ Datei wurde gespeichert: ./data/cleaned_data.csv
(1555645, 27)


,Date,Country,City,aqi,co,d,dew,humidity,mepaqi,neph,...,pressure,psi,so2,temperature,uvi,wd,wind-gust,wind-speed,Latitude,Longitude
0,2014-12-29,AT,graz,NaN,0.1,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1.6,NaN,NaN,NaN,NaN,NaN,47.06667,15.45000
1,2014-12-29,AT,innsbruck,NaN,0.1,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1.6,NaN,NaN,NaN,NaN,NaN,47.26266,11.39454
2,2014-12-29,AT,linz,NaN,0.1,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,2.1,NaN,NaN,NaN,NaN,NaN,48.30639,14.28611
3,2014-12-29,AT,salzburg,NaN,0.1,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,2.1,NaN,NaN,NaN,NaN,NaN,47.79941,13.04399
4,2014-12-29,AT,vienna,NaN,0.1,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,2.6,NaN,NaN,NaN,NaN,NaN,48.20849,16.37208


In [90]:
cities = df[['City', 'Latitude', 'Longitude']]
print(cities.head())

        City  Latitude  Longitude
0       graz  47.06667   15.45000
1  innsbruck  47.26266   11.39454
2       linz  48.30639   14.28611
3   salzburg  47.79941   13.04399
4     vienna  48.20849   16.37208


In [91]:
from meteostat import Stations

# Test mit einer Stadt
city = {'City': 'Berlin', 'Latitude': 52.5200, 'Longitude': 13.4050}

# Nächste Wetterstation suchen
stations = Stations().nearby(city['Latitude'], city['Longitude'])
station = stations.fetch(5)  # Hol dir mal die nächsten 5 Stationen zum Vergleich

print(station)


                          name country region    wmo  icao  latitude  \
id                                                                     
10389  Berlin / Alexanderplatz      DE     BE  10389  <NA>   52.5167   
10384       Berlin / Tempelhof      DE     BE  10384  EDDI   52.4667   
ETNB0   Berlin / Reinickendorf      DE     BE   <NA>  ETNB   52.5667   
10382           Berlin / Tegel      DE     BE  10382  EDDT   52.5667   
10381          Berlin / Dahlem      DE     BE  10381  <NA>   52.4667   

       longitude  elevation       timezone hourly_start hourly_end  \
id                                                                   
10389    13.4167       37.0  Europe/Berlin   1969-12-01 2011-07-31   
10384    13.4000       50.0  Europe/Berlin   1929-08-01 2025-03-02   
ETNB0    13.3167       36.0  Europe/Berlin   1991-04-21 1991-10-27   
10382    13.3167       37.0  Europe/Berlin   1936-03-10 2021-05-05   
10381    13.3000       51.0  Europe/Berlin   1951-01-01 2025-03-02   

    

In [92]:
from meteostat import Daily
from datetime import datetime

# Test mit der ersten Station
start = datetime(2023, 1, 1)
end = datetime(2023, 1, 31)

if not station.empty:
    station_id = station.index[0]
    print(f"📡 Verwende Station: {station_id}")

    # Tägliche Wetterdaten abrufen
    data = Daily(station_id, start, end)
    data = data.fetch()

    print(data.head())
else:
    print("⚠️ Keine Wetterstation gefunden!")

📡 Verwende Station: 10389
            tavg  tmin  tmax  prcp  snow   wdir  wspd  wpgt    pres  tsun
time                                                                     
2023-01-01  13.9  11.1  15.5   NaN   NaN  224.0  21.1  50.0  1014.9   NaN
2023-01-02  12.1  10.1  14.6   NaN   NaN  221.0  17.4  38.9  1016.1   NaN
2023-01-03   6.2   3.1   9.7   NaN   NaN  252.0  13.1  33.3  1026.6   NaN
2023-01-04   6.4   3.5   9.5   NaN   NaN  214.0  22.3  50.0  1016.0   NaN
2023-01-05   7.9   4.6   9.2   NaN   NaN  285.0  23.3  59.3  1012.8   NaN


In [93]:
#WEtterdaten integrieren



from meteostat import Daily, Stations
from datetime import datetime

# Städte
cities = df[['City', 'Latitude', 'Longitude']].drop_duplicates()

# Zeitspanne festlegen
start = datetime(2015, 1, 1)
end = datetime(2024, 1, 31)

# DataFrame für alle Städte vorbereiten
all_data = pd.DataFrame()

# Daten für jede Stadt abrufen und hinzufügen
for _, city in cities.iterrows():
    # Nächste Wetterstation suchen
    stations = Stations().nearby(city['Latitude'], city['Longitude'])
    station = stations.fetch(1)

    if not station.empty:
        station_id = station.index[0]

        # Tägliche Wetterdaten abrufen
        data = Daily(station_id, start, end).fetch()

        # Nan-Daten rausfiltern
        data.dropna(how='all', inplace=True)

        if not data.empty:
            # Stadtname hinzufügen
            data["City"] = city["City"]

            # Daten in den Gesamtdaten-Frame einfügen
            all_data = pd.concat([all_data, data])

    # if station.empty:
    #     print(f"⚠️ Keine Wetterstation für {city['City']} gefunden, übersprungen.")
    #     continue

    # # Tägliche Wetterdaten abrufen
    # data = Daily(station.index[0], start, end)
    # data = data.fetch()

    # if data.empty:
    #     print(f"⚠️ Keine Wetterdaten für {city['City']} gefunden, übersprungen.")
    #     continue

    # # Stadtname hinzufügen
    # data["City"] = city["City"]

    # # Daten in den Gesamtdaten-Frame einfügen
    # all_data = pd.concat([all_data, data])

# Index zurücksetzen
all_data.reset_index(inplace=True)

# Spalte time umbennen in Date
all_data.rename(columns={'time': 'Date'}, inplace=True) 

# Standardisiere den Stadtnamen für eine bessere Übereinstimmung
all_data["City"] = all_data["City"].str.lower().str.strip()

print(f"✅ Wetterdaten gesammelt für {all_data['City'].nunique()} Städte")

# # Ergebnis anzeigen
# print(all_data.head())

✅ Wetterdaten gesammelt für 577 Städte


In [94]:
#print(df[['City', 'Date']].head())
print(all_data.head())

        Date  tavg  tmin  tmax  prcp  snow  wdir  wspd  wpgt  pres  tsun  City
0 2015-01-01  -4.0  -6.4   0.4   0.0   NaN   NaN   3.5   NaN   NaN   NaN  graz
1 2015-01-02   0.1  -4.1   5.9   0.0   NaN   NaN   NaN   NaN   NaN   NaN  graz
2 2015-01-03  -0.4  -1.3   5.6   2.1   NaN   NaN   NaN   NaN   NaN   NaN  graz
3 2015-01-04   3.2   0.4   6.2   1.8   NaN   NaN  15.1   NaN   NaN   NaN  graz
4 2015-01-05   1.4  -1.2   5.0   0.0   NaN   NaN  11.1   NaN   NaN   NaN  graz


In [95]:
print(df['Date'].dtype)
print(all_data['Date'].dtype)
print(df['City'].dtype)
print(all_data['City'].dtype)

object
datetime64[ns]
object
object


In [96]:
print(df['City'].unique())
print(all_data['City'].unique())

print(set(df['Date']).difference(set(all_data['Date'])))
print(set(df['City']).difference(set(all_data['City'])))

['graz' 'innsbruck' 'linz' 'salzburg' 'vienna' 'brisbane' 'darwin'
 'hobart' 'launceston' 'melbourne' 'newcastle' 'perth' 'sydney'
 'wollongong' 'antwerpen' 'brussels' 'charleroi' 'gent' 'liège' 'namur'
 'cochabamba' 'são josé dos campos' 'são paulo' 'calgary' 'edmonton'
 'hamilton' 'kitchener' 'london' 'mississauga' 'montréal' 'ottawa'
 'québec' 'surrey' 'toronto' 'vancouver' 'victoria' 'winnipeg' 'zürich'
 'calama' 'chillán' 'concepción' 'los ángeles' 'osorno' 'quilpué'
 'rancagua' 'santiago' 'talca' 'temuco' 'valparaíso' 'anyang' 'beijing'
 'changchun' 'changsha' 'changzhou' 'chengdu' 'chongqing' 'dongguan'
 'foshan' 'fushun' 'fuzhou' 'guangzhou' 'guiyang' 'haikou' 'hangzhou'
 'harbin' 'hefei' 'hohhot' 'jieyang' 'jinan' 'kunming' 'lanzhou' 'lhasa'
 'nanchang' 'nanjing' 'nanning' 'ningbo' 'qingdao' 'qinhuangdao' 'qiqihar'
 'shanghai' 'shantou' 'shenyang' 'shenzhen' 'shijiazhuang' 'shiyan'
 'suzhou' 'taiyuan' 'tianjin' 'wuhan' 'wuxi' 'xiamen' 'xining' 'xinxiang'
 'xi’an' 'xuchang' 'yi

In [97]:

print(type(df))
print(type(all_data))

print(df.shape)
print(all_data.shape)


# Konvertiere die 'Date' Spalte in beiden DataFrames zu datetime
df['Date'] = pd.to_datetime(df['Date']).dt.strftime('%Y-%m-%d')
all_data['Date'] = pd.to_datetime(all_data['Date']).dt.strftime('%Y-%m-%d')

print(len(set(df['Date']).difference(set(all_data['Date']))))

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
(1555645, 27)
(1694854, 12)
400


In [98]:
print(all_data.sample(15))


               Date  tavg  tmin  tmax  prcp  snow   wdir  wspd  wpgt    pres  \
122825   2022-02-26  16.2  14.0  21.0   3.8   NaN  350.0  16.1   NaN  1012.8   
636597   2016-02-20   9.9   NaN  13.2   7.6   NaN    NaN   NaN   NaN     NaN   
162360   2023-05-28  22.3  11.7  28.2   0.0   NaN   42.0   5.7   NaN  1011.7   
908768   2015-04-13   9.8   4.5  15.7   NaN   NaN  245.0  14.0   NaN  1024.2   
744755   2017-02-09  -1.2   NaN   NaN   NaN   NaN    NaN   NaN   NaN     NaN   
125583   2023-07-21  10.8   7.5  13.0  55.2   NaN   36.0   8.9   NaN  1013.1   
1538783  2015-09-18  19.6   NaN  27.1   NaN   NaN    NaN   NaN   NaN     NaN   
854524   2016-05-18  13.6   8.3  18.7   0.4   0.0  187.0  10.8   NaN  1008.6   
1259735  2016-11-22  -0.1  -3.0   4.0   1.5   NaN  265.0  14.4   NaN     NaN   
381114   2022-04-14  17.2  14.2  21.8   0.0   NaN  323.0  14.3   NaN  1015.1   
1155836  2023-06-02  14.4   9.9  20.8   0.5   NaN  299.0   7.7   NaN  1001.4   
1166226  2017-03-09   7.8   0.0  16.0   

In [99]:
# Funktion für Übersicht über dtypes, missing values, unique values etc.
def overview(df):
    '''
    Erstelle einen Überblick über einige Eigenschaften der Spalten eines DataFrames.
    VARs
        df: Der zu betrachtende DataFrame
    RETURNS:
        None
    '''
    display(pd.DataFrame({'dtype': df.dtypes,
                          'total': df.count(),
                          'missing': df.isna().sum(),
                          'missing%': df.isna().mean()*100,
                          'n_uniques': df.nunique(),
                          'uniques%': df.nunique()/df.shape[0]*100,
                          'uniques': [df[col].unique() for col in df.columns]
                         }))
overview(all_data)

,dtype,total,missing,missing%,n_uniques,uniques%,uniques
Date,object,1694854,0,0.000000,3318,0.195769,"[2015-01-01, 2015-01-02, 2015-01-03, 2015-01-0..."
tavg,float64,1665924,28930,1.706932,802,0.047320,"[-4.0, 0.1, -0.4, 3.2, 1.4, 0.3, -2.9, -3.2, 1..."
tmin,float64,1607691,87163,5.142803,785,0.046317,"[-6.4, -4.1, -1.3, 0.4, -1.2, -0.4, -1.6, -4.9..."
tmax,float64,1606610,88244,5.206584,851,0.050211,"[0.4, 5.9, 5.6, 6.2, 5.0, 4.6, -0.2, 0.8, 5.4,..."
prcp,float64,1150115,544739,32.140763,1447,0.085376,"[0.0, 2.1, 1.8, 0.8, 3.7, 0.1, 3.3, 10.1, 2.4,..."
snow,float64,159456,1535398,90.591756,174,0.010266,"[nan, 20.0, 10.0, 30.0, 61.0, 79.0, 41.0, 140...."
wdir,float64,1246621,448233,26.446703,361,0.021300,"[nan, 335.0, 340.0, 320.0, 336.0, 299.0, 314.0..."
wspd,float64,1385729,309125,18.239034,677,0.039944,"[3.5, nan, 15.1, 11.1, 6.8, 4.2, 8.6, 9.0, 5.2..."
wpgt,float64,282851,1412003,83.311188,496,0.029265,"[nan, 24.1, 20.4, 50.0, 25.9, 22.2, 29.6, 27.8..."
pres,float64,1306292,388562,22.925987,974,0.057468,"[nan, 1018.5, 1017.4, 1015.1, 1015.3, 1017.3, ..."


In [100]:
all_data.columns

Index(['Date', 'tavg', 'tmin', 'tmax', 'prcp', 'snow', 'wdir', 'wspd', 'wpgt',
       'pres', 'tsun', 'City'],
      dtype='object')

In [101]:
# Berechne den Anteil der NaN-Werte pro Spalte
missing_percentage = all_data.isna().mean() * 100

# Lösche die Spalten, bei denen der Anteil an NaN-Werten größer als 80% ist
all_data = all_data.loc[:, missing_percentage <= 80]

In [102]:
all_data.columns

Index(['Date', 'tavg', 'tmin', 'tmax', 'prcp', 'wdir', 'wspd', 'pres', 'City'], dtype='object')

In [103]:


# Zusammenführen der beiden DataFrames über "City"
df = pd.merge(df, all_data, on=["City", 'Date'],how="left")
print(df.shape)
df.sample(30)

(1565693, 34)


,Date,Country,City,aqi,co,d,dew,humidity,mepaqi,neph,...,wind-speed,Latitude,Longitude,tavg,tmin,tmax,prcp,wdir,wspd,pres
431139,2019-05-21,GB,belfast,NaN,0.8,NaN,6.0,70.0,NaN,NaN,...,2.1,54.59682,-5.92541,10.7,7.0,15.0,NaN,NaN,10.7,1019.8
164243,2016-06-19,US,oklahoma city,NaN,4.5,NaN,NaN,NaN,NaN,NaN,...,NaN,35.46756,-97.51643,26.4,21.1,32.8,0.0,147.0,15.8,1018.7
1200813,2023-02-23,KW,ḩawallī,NaN,6.5,NaN,NaN,74.6,NaN,NaN,...,2.1,29.33278,48.02861,18.6,13.0,23.9,0.2,65.0,10.4,1011.5
1051636,2022-06-09,CL,concepción,NaN,14.4,NaN,7.0,71.0,NaN,NaN,...,3.6,-36.82699,-73.04977,11.2,8.7,14.2,0.7,141.0,12.2,1025.9
1441998,2024-07-29,CA,hamilton,NaN,4.6,NaN,19.0,79.5,NaN,NaN,...,1.5,43.25011,-79.84963,NaN,NaN,NaN,NaN,NaN,NaN,NaN
538592,2019-11-19,ES,santander,NaN,0.1,NaN,6.5,84.0,NaN,NaN,...,2.0,43.46472,-3.80444,10.5,7.0,13.7,0.0,160.0,12.4,1015.8
932014,2021-08-28,JO,zarqa,NaN,39.7,NaN,13.0,31.0,NaN,NaN,...,3.0,32.07275,36.08796,31.0,25.8,36.0,NaN,304.0,7.7,1010.1
326953,2018-05-12,US,salem,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,44.94290,-123.03510,15.1,9.4,27.2,0.0,NaN,9.4,1016.9
516488,2019-10-13,CN,xuchang,NaN,8.2,NaN,12.0,60.0,NaN,NaN,...,1.5,34.03189,113.86299,18.4,14.5,23.0,0.3,19.0,6.8,1023.2
544775,2019-11-29,PT,lisbon,NaN,NaN,NaN,16.0,93.0,NaN,NaN,...,3.6,38.71667,-9.13333,16.9,16.7,18.4,0.0,NaN,NaN,NaN


In [104]:
df.columns

Index(['Date', 'Country', 'City', 'aqi', 'co', 'd', 'dew', 'humidity',
       'mepaqi', 'neph', 'no2', 'o3', 'pm1', 'pm10', 'pm25', 'pol',
       'precipitation', 'pressure', 'psi', 'so2', 'temperature', 'uvi', 'wd',
       'wind-gust', 'wind-speed', 'Latitude', 'Longitude', 'tavg', 'tmin',
       'tmax', 'prcp', 'wdir', 'wspd', 'pres'],
      dtype='object')

In [105]:
print(df.shape)
df.isna().sum()

(1565693, 34)


Date                   0
Country                0
City                   0
aqi              1525974
co                589490
d                1563662
dew               636182
humidity          353744
mepaqi           1562917
neph             1561097
no2               271151
o3                355363
pm1              1559876
pm10              293827
pm25              242841
pol              1560903
precipitation    1436151
pressure          358256
psi              1565330
so2               493890
temperature       352069
uvi              1536624
wd               1485393
wind-gust         870145
wind-speed        376919
Latitude            2827
Longitude           2827
tavg              439379
tmin              468540
tmax              470678
prcp              769831
wdir              656078
wspd              588808
pres              623958
dtype: int64